# Recommendations with only implicit feedback

In [1]:
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from sklearn.linear_model import ElasticNet
import random

## Explore the data

CiteULike data set

data set downloaded from https://github.com/js05212/citeulike-a

data set consists of article collections of users, in user-article pairs

In [2]:
likes_per_user={}
user=0
item_list=[]
with open("/home/henning/Downloads/citeulike-a-master/users.dat") as f:
    for line in f.readlines():
        likes_per_user[user]=[int(item) for item in line.split()]
        user+=1
users=range(user)
user_num=user

In [3]:
def compute_per_item(likes_per_user):
    per_item={}
    for user in users:
        likes=likes_per_user[user]
        for item in likes:
            liking_users=per_item.get(item,[])
            liking_users.append(user)
            per_item[item]=liking_users            
    return per_item

In [4]:
likes_per_item=compute_per_item(likes_per_user)
item_num=len(likes_per_item.keys())
item_list=list(likes_per_item.keys())
item_list.sort()
item_list[0], item_list[-1],item_num

(0, 16979, 16980)

Good! Items are just 0,...,16979

In [5]:
user_num

5551

In [6]:
# let's compute statistics on number of items liked by users
# min, max, average
like_nums=[len(likes_per_user[user]) for user in users]
min(like_nums),max(like_nums),sum(like_nums)/len(users)

(11, 404, 37.927760763826335)

## Training and test set

In [7]:
# put five liked items per user in test set
test={}
train={}
for user in users:
    likes=likes_per_user[user]
    likes=shuffle(likes)
    test[user]=likes[:5]
    train[user]=likes[5:]

As metric we use hit rate: the average number of good recommendations in the recommendation list per user

In [8]:
def hit_count(user,recommendations):
    return len([item for item in recommendations if item in test[user]])

def hit_rate(recommendations_per_user):
    total_hits=sum([hit_count(user,recommendations_per_user[user]) for user in users])
    return total_hits/len(users)

In [9]:
# let's test this
always_42=dict([(user,[42]) for user in users])
print("recommend 42 to all: {}".format(hit_rate(always_42)))

# let's also cheat
cheat=dict([(user,[test[user][0]]) for user in users])
print("cheat: {}".format(hit_rate(cheat)))

recommend 42 to all: 0.003422806701495226
cheat: 1.0


In [10]:
# let's fix length of recommendation list
rec_len=10

## Baseline

As baseline we will recommend the most popular items to everybody. Any decent recommendation algorithm should beat the baseline.

In [11]:
from operator import itemgetter
train_per_item=compute_per_item(train)
by_popularity=sorted([(len(train_per_item[item]),item) for item in train_per_item.keys()],key=itemgetter(0),reverse=True)
most_popular=[item for (_,item) in by_popularity[:rec_len]]
most_popular

[14, 15, 3981, 17, 28, 16, 10, 16045, 11, 13]

In [12]:
# recommend the most popular items to everybody
hit_rate(dict([(user,most_popular) for user in users]))

0.159430733201225

In [13]:
# next try, do not recommend anything that the user already likes
def top_popular(user):
    result=[]
    for (_,item) in by_popularity:
        if item not in train[user]:
            result.append(item)
            if len(result)>=rec_len:
                return result

hit_rate(dict([(user,top_popular(user)) for user in users]))

0.16213294901819492

...a tiny bit better

## base class score recommender

Define a base class for recommendation algorithms that are based on scores. It just does two things: computes the user-item like matrix, stored as a sparse matrix; and it processes scores to output the recommendation list, where it, in particular, filters out likes that are already known from the training set. 

In [14]:
class ScoreRec:
    def __init__(self,rec_len,user_num,item_num):
        """
        rec_len: length of recommendation list
        user_num: the number of users; we expect the users to be range(user_num)
        item_num: the number of items; we expect the items to be range(item_num)
        """
        self.rec_len=rec_len
        self.user_num=user_num
        self.item_num=item_num

    def compute_train_matrix(self):
        """compute and store user-item like matrix; matrix is stored as a sparse matrix"""
        train_likes=np.array([(user,item) for user in self.likes_per_user.keys() for item in self.likes_per_user[user]])
        rows=train_likes[:,0].reshape(-1,)
        cols=train_likes[:,1].reshape(-1,)
        # we assume implicit feedback, thus all values in the matrix are ones
        # why the many parentheses? Because scipy sparse API is weird
        self.train_matrix=scipy.sparse.csr_matrix((np.ones(len(rows)),(rows,cols)),
                                                  shape=(self.user_num,self.item_num))
        
    def filter_by_scores(self,user,scores):
        """rank by scores but first filter out the items already in training set"""
        # set scores of items already in training set to -infinity to prevent them from being selected
        scores[self.likes_per_user[user]]=np.NINF # -infinity
        # see numpy argpartition, picks arguments of highest scores
        return np.argpartition(scores,-self.rec_len)[-self.rec_len:]
        
    def compute_scores(self,user):
        # implement in child classes
        pass

    def predict_per_user(self,user):
        scores=self.compute_scores(user)
        return self.filter_by_scores(user,scores)
    
    def predict(self,users):
        """predict expects a list of users; class cannot handle users not seen during training"""
        return dict([(user,self.predict_per_user(user)) for user in users])

## SLIM

algorithm from 

SLIM: Sparse Linear Methods for Top-N Recommender System, X. Ning and G. Karypis
(2011)

In [15]:
class SLIM(ScoreRec):
    def __init__(self,alpha,beta,rec_len,user_num,item_num,positive_weights=False):
        """
        regularisation: alpha L2 + beta L1
        rec_len: length of recommendation list
        user_num,item_num: number of users, items -- we expect range(user_num) and 
        range(item_num) as set of users and items
        current implementation works only with implicit feedback
        """
        # elastic net computes regularisation strength differently, see docu
        super().__init__(rec_len,user_num,item_num)
        alp=alpha+beta  
        l1_ratio=alpha/(alpha+beta)
        self.net=ElasticNet(alpha=alp,l1_ratio=l1_ratio,fit_intercept=False,
                            max_iter=100,copy_X=False,
                            tol=0.0001,precompute=True,selection='random',
                            positive=positive_weights)  
                
    def __compute_weight_vec(self,item):
        """do elastic net regression for weight vector corresponding to item"""
        target_col=self.train_matrix.getcol(item).toarray().reshape(-1,)
        T=scipy.sparse.csr_matrix((target_col,(list(range(self.user_num)),[item]*self.user_num)),
                                  shape=self.train_matrix.shape)
        # we zero out the column corresponding to item, that's what T is for
        self.net.fit(self.train_matrix-T,target_col)
        # store learned non-zero weights in lists: row_index,col_index,weight
        # as this is the format that scipy sparse expects 
        for j,value in enumerate(self.net.coef_):
            if value!=0:
                # we store transpose of weight matrix
                self.weight_rows.append(j)
                self.weight_cols.append(item)
                self.weight_values.append(value)
        
    def fit(self,likes_per_user):
        """expects a dictionary with keys=users, values=list of liked items"""
        self.likes_per_user=likes_per_user
        self.compute_train_matrix()
        self.weight_rows=[]
        self.weight_cols=[]
        self.weight_values=[]
        for item in range(self.item_num):
            self.__compute_weight_vec(item)
        self.weight_matrix=scipy.sparse.csr_matrix((self.weight_values,(self.weight_rows,self.weight_cols)),
                                           shape=(self.item_num,self.item_num))
        # free memory
        del(self.weight_rows)
        del(self.weight_cols)
        del(self.weight_values)

    def compute_scores(self,user):
        user_row=self.train_matrix.getrow(user).toarray()
        user_row=user_row.reshape(-1,)
        # prediction for user as follows: weight matrix * transpose of user row
        # for this we store transpose of original weight matrix
        return self.weight_matrix.dot(user_row)  # multiplication returns numpy array

In [16]:
# Regularisation strength, alpha, beta, would need to be fine-tuned
# with a validation set. I was sloppy and simply tried out three sets
# of values. This is not how it should be done, but running fit takes
# a couple of minutes and I am not overly patient.
slim=SLIM(0.001,0.001,rec_len,user_num,item_num)
slim.fit(train)
print("number of non-zero weights: ",slim.weight_matrix.nnz)

/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Object

/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Object

number of non-zero weights:  45294


In [17]:
preds=slim.predict(users)
print("hit rate: ",hit_rate(preds))

hit rate:  0.44640605296343


In [18]:
# let's do the same thing again, only this time, force weights to be positive
slim=SLIM(0.001,0.001,rec_len,user_num,item_num,positive_weights=True)
slim.fit(train)
print("number of non-zero weights: ",slim.weight_matrix.nnz)

/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Object

/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  max_iter, tol, rng, random, positive)
/home/henning/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Object

number of non-zero weights:  45287


In [19]:
preds=slim.predict(users)
print("hit rate: ",hit_rate(preds))

hit rate:  0.44622590524229866


## Recommendation by random walks

Random walks in recommender systems: exact computation and simulation, C. Cooper,
S.H. Lee, T. Radzik and Y. Siantos (2014)

In [20]:
class RndWalkRec(ScoreRec):
    """recommendations by random walks -- can only handle implicit feedback in current implementation"""

    def compute_prob_matrix(self,like_matrix):
        """
        simply normalise every row; will fail to produce probabilities if there are negative entries
        outputs new sparse matrix
        """
        probabilities=np.zeros(like_matrix.get_shape()[0])
        for row_num in range(like_matrix.get_shape()[0]):
            row_sum=like_matrix.getrow(row_num).sum()
            if row_sum>0:
                probabilities[row_num]=1/row_sum
            else:
                probabilities[row_num]=0
        return scipy.sparse.diags(probabilities).dot(like_matrix)
    
    def fit(self,likes_per_user):
        self.likes_per_user=likes_per_user
        self.compute_train_matrix()
        A=self.compute_prob_matrix(self.train_matrix)
        B=self.compute_prob_matrix(self.train_matrix.transpose())
        self.P3=A.dot(B.dot(A))
    
    def compute_scores(self,user):
        return self.P3.getrow(user).toarray().reshape(-1,)

In [21]:
walker=RndWalkRec(rec_len,user_num,item_num)
walker.fit(train)

In [22]:
preds=walker.predict(users)
print("hit rate: ",hit_rate(preds))

hit rate:  1.0679156908665106


## Simulated walks

Simulate random walks instead of calculating power of matrix. 

In [23]:
class SimulRndWalkRec(ScoreRec):

    def __init__(self,walk_num,rec_len,user_num,item_num):
        """
        walk_num: number of random walks simulated per user
        rec_len: length of recommendation list
        user_num,item_num: number of users, items -- we expect range(user_num) and 
        range(item_num) as set of users and items
        current implementation works only with implicit feedback
        """
        super().__init__(rec_len,user_num,item_num)
        self.walk_num=walk_num

    def fit(self,likes_per_user):
        self.likes_per_user=likes_per_user
        self.likes_per_item=compute_per_item(likes_per_user)
        
    def rnd_user_path(self,user):
        """do random walk for single user"""
        item=random.choice(self.likes_per_user[user])
        user2=random.choice(self.likes_per_item[item])
        return random.choice(self.likes_per_user[user2])
    
    def compute_scores(self,user):
        scores=np.zeros(self.item_num)
        for _ in range(self.walk_num):
            scores[self.rnd_user_path(user)]+=1
        return scores

In [24]:
simu=SimulRndWalkRec(1000,rec_len,user_num,item_num)
simu.fit(train)
preds=simu.predict(users)
print("hit rate: ",hit_rate(preds))

hit rate:  0.6960907944514502


In [25]:
simu=SimulRndWalkRec(2000,rec_len,user_num,item_num)
simu.fit(train)
preds=simu.predict(users)
print("hit rate: ",hit_rate(preds))

hit rate:  0.8248964150603495
